# Download json records for all published datasets

This should work for any socrata portal

In [24]:
#import the required libraries
import csv
import requests
import json as json
import datetime

## Set up the code for your portal of interest

In [ ]:
targeturl='https://data.wa.gov/' #change this to the SOCRATA portal you want to target, don't forget ending /
descriptor='WA'   #change this to a recognizable descriptor for yourself- used in csv filename

### Create Dictionary of dcat records
Use the Socrata API to get basic metadata for the public assets

In [4]:
r=requests.get(targeturl+"api/dcat.json") #build string according to SOCRATA's convention to get public data assets
j=json.loads(r.text) #parse the json into a list named j

In [150]:
#j[0] #view the first record

### Test Pit

Ignore this block.  Use it to test out things or store bits of unused code

In [ ]:
#The indexes for i (below) are much more limited than the indexes for j
test=[] #test list to fill

for i in j:
    #for x in i:
        if i.get('format') == 'text/csv':
            t = i.get('identifier')
            test.append(t)
            #print(i.get('format'))
test.sort()
print(test[0:20])


#print(len(test))
#j[0].keys() #Shows keys
#type(j['identifier'])
#print(j[0]['format'])



### Create a list of all the uids that are datasets

In [38]:
masterlist=[] #dim masterlist as a empty list

for i in j:  #j is the dictionary of parsed json
        if len(i['identifier'])==9: #Basically, is the identifier legitimate?
            if i.get('format')=='text/csv':  #select only datasets (not pdfs or external links)
                uid=i['identifier']
                masterlist.append(uid)


masterlist.sort() #sort masterlist
print("List built:", len(masterlist),"elements") #print how many elements are in masterlist
#print(masterlist[0:100]) See the first hundred if you want to check

List built: 551 elements


### If you want to see the list of urls for metadata, run this code

In [37]:
for m in masterlist:
    print(targeturl+'views/'+m+'.json') #build string according to SOCRATA's convention


## Create a csv of the metadata

Basic order of operations:
Set up csv file with titles
For loop to cycle through masterlist of dataset ids from above

    Retrieve json file
    Count the number of columns
    Count the number of non empty column desctiptions
    Write elements to csv file, convert the dates, and tack on the two count values

Here are the columns: 'id','name','description','category','downloadCount','viewCount','licenseId','publicationDate','rowsUpdateAt','tags','rowLabel','numberColumns','numColumnDescriptions'

In [23]:
metadata=open(descriptor+' - METADATA.csv', 'w', newline='')
csv.writer(metadata).writerow(['id','name','description','category','downloadCount','viewCount','licenseId','publicationDate','rowsUpdateAt','tags'])

metadata=open(descriptor+' - METADATA.csv', 'w', newline='')
csv.writer(metadata).writerow(['id','name','description','category','downloadCount','viewCount','licenseId','publicationDate','rowsUpdateAt','tags','rowLabel','numberColumns','numColumnDescriptions'])
for s in masterlist:
    s=requests.get(targeturl+'views/'+s+'.json') #build string according to SOCRATA's convention
    r=json.loads(s.text)
    
    colnum = len(r['columns']) #Get number of columns
    
    count = 0
    for q in r['columns']:  #Tally the number of col descriptions that contain a description
        if q.get('description','') != '': #If the description key doesn't exist, default value is '' (don't know why some don't have the key)
            count += 1
    coldesc = count
    #write the values to the csv file. Dates in json files are seconds from jan 1 1970 so datetime.datetime.fromtimestamp converts
    csv.writer(metadata).writerow([r['id'].encode("utf-8"),r['name'].encode("utf-8"),r.get('description','N/A'),r.get('category','N/A'),r['downloadCount'],r['viewCount'],r.get('LicenseID','N/A'),datetime.datetime.fromtimestamp(r['publicationDate']),datetime.datetime.fromtimestamp(r['rowsUpdatedAt']),r.get('tags','N/A'),r['metadata'].get('rowLabel','N/A'),colnum,coldesc]) #write one line to csv file

metadata.close() #Close the output file, release all locks
print(len(j)-1,"of",len(j)-1,"rows written, 0 remaining")

916 of 916 rows written, 0 remaining
